In [2]:
from datetime import datetime, timedelta
from json import loads, JSONDecodeError
from logging import (
    basicConfig,
    CRITICAL,
    ERROR,
    FileHandler,
    getLogger,
    INFO,
    log,
    StreamHandler,
)
from os import getenv, makedirs, path
from re import findall
from time import localtime, sleep, strftime, time
from traceback import TracebackException

from dotenv import load_dotenv
from openpyxl import load_workbook, Workbook
from pandas import DataFrame
from seleniumwire import webdriver
from seleniumwire.utils import decode
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    ElementNotInteractableException,
)
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.remote_connection import LOGGER as seleniumLogger
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib3.connectionpool import log as urllibLogger
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
class Errores:
    """
    Representa a los errores ocurridos durante la ejecución de un scraper

    ...

    Attributes
    ----------
    errores : dict
        Conjunto de datos que contiene toda información de los errores ocurridos durante la ejecución del scraper

    Methods
    -------
    agregar_error(error, enlace):
        Agrega la información de un error al diccionario de datos errores
    """

    def __init__(self):
        """
        Genera todos los atributos para el objeto Errores
        """
        self._errores = {
            "Clase": [],
            "Mensaje": [],
            "Linea de Error": [],
            "Codigo Error": [],
            "Publicacion": [],
        }

    @property
    def errores(self):
        """Retorna el valor actual del diccionario de datos errores"""
        return self._errores

    def agregar_error(self, error, enlace):
        """
        Agrega la información de un error al diccionario de datos errores

        Parameters
        ----------
        error: Exception
            Objeto de tipo excepción ocurrida durante la ejecución del scraper
        enlace: str
            Enlace de la publicación de la página facebook marketplace

        Returns
        -------
        None
        """
        log(ERROR, f"Error:\n{error}")
        traceback_error = TracebackException.from_exception(error)
        error_stack = traceback_error.stack[0]
        self._errores["Clase"].append(traceback_error.exc_type)
        self._errores["Mensaje"].append(traceback_error._str)
        self._errores["Linea de Error"].append(error_stack.lineno)
        self._errores["Codigo Error"].append(error_stack.line)
        self._errores["Publicacion"].append(enlace)

In [5]:
class Dataset:
    """
    Representa al conjunto de datos generado por el scraper

    ...

    Attributes
    ----------
    dataset : dict
        Conjunto de datos que contiene toda información extraída de una categoría de la página de facebook marketplace

    Methods
    -------
    agregar_data():
        Agrega la información de una publicación al diccionario de datos dataset
    """

    def __init__(self):
        """
        Genera todos los atributos para el objeto Dataset
        """
        self._dataset = {
            "Fecha Extraccion": [],
            "titulo_marketplace": [],
            "tiempo_creacion": [],
            "tipo_delivery": [],
            "descripcion": [],
            "disponible": [],
            "vendido": [],
            "fecha_union_vendedor": [],
            "cantidad": [],
            "precio": [],
            "tipo_moneda": [],
            "amount_with_concurrency": [],
            "latitud": [],
            "longitud": [],
            "locacion": [],
            "locacion_id": [],
            "name_vendedor": [],
            "tipo_vendedor": [],
            "id_vendedor": [],
            "enlace": [],
        }

    @property
    def dataset(self):
        """Retorna el valor actual del diccionario de datos dataset"""
        return self._dataset

    def agregar_data(self, item, fecha_extraccion, enlace):
        """
        Agrega la información de una publicación al dataset

        Parameters
        ----------
        item: dict
            Conjunto de datos que contiene toda la información de una publicación
        fecha_extraccion: str
            Fecha actual en la que se creó una publicación en formato %d/%m/%Y
        enlace: str
            Enlace de la publicación de la página facebook marketplace

        Returns
        -------
        None
        """
        self._dataset["titulo_marketplace"].append(
            item.get("marketplace_listing_title")
        )
        self._dataset["tiempo_creacion"].append(item.get("creation_time"))
        self._dataset["disponible"].append(item.get("is_live"))
        self._dataset["vendido"].append(item.get("is_sold"))
        self._dataset["cantidad"].append(item.get("listing_inventory_type"))
        self._dataset["name_vendedor"].append(
            item.get("story").get("actors")[0].get("name")
        )
        self._dataset["tipo_vendedor"].append(
            item.get("story").get("actors")[0]["__typename"]
        )
        self._dataset["id_vendedor"].append(item.get("story").get("actors")[0]["id"])
        self._dataset["locacion_id"].append(item.get("location_vanity_or_id"))
        self._dataset["latitud"].append(item.get("location", {}).get("latitude"))
        self._dataset["longitud"].append(item.get("location", {}).get("longitude"))
        self._dataset["precio"].append(item.get("listing_price", {}).get("amount"))
        self._dataset["tipo_moneda"].append(
            item.get("listing_price", {}).get("currency")
        )
        self._dataset["amount_with_concurrency"].append(
            item.get("listing_price", {}).get("amount_with_offset_in_currency")
        )
        self._dataset["tipo_delivery"].append(item.get("delivery_types", [None])[0])
        self._dataset["descripcion"].append(
            item.get("redacted_description", {}).get("text")
        )
        self._dataset["fecha_union_vendedor"].append(
            item.get("marketplace_listing_seller", {}).get("join_time")
        )
        data = item.get("location_text", {})
        if data:
            data = data.get("text")
        self._dataset["locacion"].append(data)
        self._dataset["Fecha Extraccion"].append(fecha_extraccion)
        self._dataset["enlace"].append(enlace)

In [6]:
class Tiempo:
    """
    Representa el tiempo que se demora el scraper en extraer la información

    ...

    Attributes
    ----------
    start : float
        Hora actual en segundos
    hora_inicio : str
        Hora de inicio de la ejecución del scraper en formato %H:%M:%S
    fecha : str
        Fecha de las publicaciones a extraer en formato %d/%m/%Y
    hora_fin : str
        Hora de término de la ejecución del scraper en formato %H:%M:%S
    cantidad : int
        Cantidad de publicaciones extraídas de la página de facebook marketplace
    cantidad_real: int
        Cantidad real de publicaciones extraídas de la página de facebook marketplace
    tiempo : str
        Tiempo de ejecución del scraper en formato %d days, %H:%M:%S
    productos_por_min : float
        Cantidad de publicaciones que puede extraer el scraper en un minuto
    productos_por_min_real : float
        Cantidad real de publicaciones que puede extraer el scraper en un minuto
    num_error : int
        Cantidad de errores ocurridos durante la ejecución del scraper

    Methods
    -------
    set_param_final():
        Establece los parámetros finales cuando se termina de ejecutar el scraper
    """

    def __init__(self, fecha_actual):
        """
        Genera todos los atributos para el objeto Tiempo

        Parameters
        ----------
        fecha_actual: str
            Fecha en la que se ejecuta el scraper
        """
        self._start = time()
        self._hora_inicio = strftime("%H:%M:%S", localtime(self._start))
        log(INFO, f"Hora de inicio: {self._hora_inicio}")
        self._fecha = fecha_actual.strftime("%d/%m/%Y")
        self._hora_fin = None
        self._cantidad = None
        self._cantidad_real = None
        self._tiempo = None
        self._productos_por_min = None
        self._productos_por_min_real = None
        self._num_error = None

    @property
    def cantidad(self):
        """Retorna el valor actual o asigna un nuevo valor del atributo cantidad"""
        return self._cantidad

    @property
    def cantidad_real(self):
        """Retorna el valor actual o asigna un nuevo valor del atributo cantidad_real"""
        return self._cantidad_real

    @property
    def fecha(self):
        """Retorna el valor actual del atributo fecha"""
        return self._fecha

    @property
    def num_error(self):
        """Retorna el valor actual o asigna un nuevo valor del atributo num_error"""
        return self._num_error

    @cantidad.setter
    def cantidad(self, cantidad):
        self._cantidad = cantidad

    @cantidad_real.setter
    def cantidad_real(self, cantidad_real):
        self._cantidad_real = cantidad_real

    @num_error.setter
    def num_error(self, num_error):
        self._num_error = num_error

    def set_param_final(self):
        """
        Establece parametros finales para medir el tiempo de ejecución del scraper

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        end = time()
        self._hora_fin = strftime("%H:%M:%S", localtime(end))
        log(INFO, f"Productos Extraídos: {self._cantidad}")
        log(INFO, f"Hora Fin: {self._hora_fin}")
        total = end - self._start
        self._tiempo = str(timedelta(seconds=total)).split(".")[0]
        self._productos_por_min = round(self._cantidad / (total / 60), 2)
        self._productos_por_min_real = round(self._cantidad_real / (total / 60), 2)

In [7]:
class ScraperFb:
    """
    Representa a un bot para hacer web scraping en fb marketplace

    ...

    Attributes
    ----------
    tiempo : Tiempo
        Objeto de la clase Tiempo que maneja información del tiempo de ejecución del scraper
    driver: webdriver.Chrome
        Objeto de la clase webdriver que maneja un navegador para hacer web scraping
    wait : WebDriverWait
        Objeto de la clase WebDriverWait que maneja el Tiempo de espera durante la ejecución del scraper
    errores : Errores
        Objeto de la clase Errores que maneja información de los errores ocurridos durante la ejecución del scraper
    data : Dataset
        Objeto de la clase Dataset que maneja información de las publicaciones extraídas por el scraper

    Methods
    -------
    iniciar_sesion():
        Iniciar sesión en facebook usando un usuario y contraseña
    mapear_datos(url):
        Mapea y extrae los datos de las publicaciones de una categoría
    guardar_datos(dataset, filetype, folder, filename):
        Guarda los datos o errores obtenidos durante la ejecución del scraper
    guardar_tiempos(filename, sheet_name):
        Guarda la información del tiempo de ejecución del scraper
    """

    def __init__(self, fecha_actual):
        """
        Genera todos los atributos para el objeto ScraperFb

        Parameters
        ----------
        fecha_actual: str
            Fecha en la que se ejecuta el scraper
        """
        log(INFO, "Inicializando scraper")
        self._tiempo = Tiempo(fecha_actual)
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        self._driver = webdriver.Chrome(
            chrome_options=chrome_options,
            service=Service(ChromeDriverManager().install()),
        )
        self._wait = WebDriverWait(self._driver, 10)
        self._errores = Errores()
        self._data = Dataset()

    @property
    def data(self):
        """Retorna el valor actual del atributo data"""
        return self._data

    @property
    def errores(self):
        """Retorna el valor actual del atributo errores"""
        return self._errores

    def iniciar_sesion(self):
        """
        Inicia sesión en una página web usando un usuario y contraseña

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        log(INFO, "Iniciando sesión")
        self._driver.get("https://www.facebook.com/")
        self._driver.maximize_window()
        username = self._wait.until(EC.presence_of_element_located((By.ID, "email")))
        password = self._wait.until(EC.presence_of_element_located((By.ID, "pass")))
        username.clear()
        password.clear()
        username.send_keys(getenv("FB_USERNAME"))
        password.send_keys(getenv("FB_PASSWORD"))
        self._wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='login']"))
        ).click()
        sleep(10)
        log(INFO, "Inicio de sesión con éxito")

    def obtener_publicaciones(self, selector, xpath):
        """
        Retornar una lista de publicaciones visibles con respecto a una categoría en facebook marketplace

        Parameters
        ----------
        selector: str
            Selector a ser usado para localizar las publicaciones
        xpath: str
            Ruta de las publicaciones a ser usado por el selector

        Returns
        -------
        list
        """
        return self._driver.find_elements(selector, xpath)

    def mapear_datos(self, url):
        """
        Mapea y extrae los datos de las publicaciones de una categoría

        Parameters
        ----------
        url: str
            Link de la página de una categoría en facebook marketplace

        Returns
        -------
        None
        """
        log(INFO, "Accediendo a la URL")
        self._driver.execute_script("window.open('about:blank', 'newtab');")
        self._driver.switch_to.window("newtab")
        self._driver.get(url)
        sleep(8)

        log(INFO, "Mapeando Publicaciones")
        ropa = self.obtener_publicaciones(
            By.XPATH, '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]'
        )

        log(INFO, "Creando variables")
        # Enteros que hacen referencia a la fecha en que se postea una publicación y en la que se extrae la información
        fecha_publicacion = fecha_extraccion = int(
            datetime.strptime(self._tiempo.fecha, "%d/%m/%Y").timestamp()
        )
        # Entero que hace referencia al día siguiente de la fecha en la que se extrae la información
        fecha_flag = fecha_extraccion + 86400
        # Cuenta la cantidad de publicaciones que mapea el scraper
        i = 0
        # Cuenta la cantidad de errores ocurridos durante la ejecución del mapeo del scraper
        e = 0
        # Flag para manejar el intento de veces que se
        f = 0
        while fecha_publicacion >= fecha_extraccion:
            try:
                log(INFO, f"Scrapeando item {i + 1}")
                # Eliminar de la memoria requests innecesarios
                del self._driver.requests
                # Link de la publicación de facebook
                enlace = findall(
                    "(.*)\/\?",
                    ropa[i]
                    .find_element(By.XPATH, ".//ancestor::a")
                    .get_attribute("href"),
                )[0]
                # Dar click a la publicación de facebook
                ropa[i].click()
                sleep(5)

                for request in self._driver.requests:
                    # Validar si la api es de graphql
                    if not request.response or "graphql" not in request.url:
                        continue
                    # Obtener la respuesta de la api en bytes
                    body = decode(
                        request.response.body,
                        request.response.headers.get("Content-Encoding", "identity"),
                    )
                    # Decodificar la respuesta a utf-8
                    decoded_body = body.decode("utf-8")

                    # Validar si la respuesta decodificada es la deseada
                    if decoded_body.find('"extensions":{"prefetch_uris_v2"') == -1:
                        continue

                    # Convertir al formato json la respuesta decodificada anteriormente
                    json_data = loads(decoded_body)
                    # Extraer la fecha de publicación
                    fecha_publicacion = json_data["data"]["viewer"][
                        "marketplace_product_details_page"
                    ]["target"]["creation_time"]

                    # Validar si la fecha de publicación corresponda a la deseada
                    if fecha_publicacion < fecha_flag:
                        # Diccionario que contiene toda la información de la publicación
                        dato = json_data["data"]["viewer"][
                            "marketplace_product_details_page"
                        ]["target"]
                        log(INFO, f"{dato['marketplace_listing_title']}")
                        self._data.agregar_data(dato, self._tiempo.fecha, enlace)
                        log(INFO, f"Item {i + 1} scrapeado con éxito")

                    break
                # Regresar al inicio donde se encuentran todas las publicaciones de facebook
                self._driver.execute_script("window.history.go(-1)")

            except (
                NoSuchElementException,
                ElementNotInteractableException,
                StaleElementReferenceException,
            ) as error:
                enlace = None
                self._errores.agregar_error(error, enlace)
                e += 1

            except (KeyError, JSONDecodeError) as error:
                self._errores.agregar_error(error, enlace)
                e += 1
                self._driver.execute_script("window.history.go(-1)")

            except Exception as error:
                self._errores.agregar_error(error, enlace)
                e += 1
                i += 1
                log(CRITICAL, "Se detuvo inesperadamente el programa")
                log(CRITICAL, f"Causa:\n{error}")
                break

            finally:
                i += 1
                if i == len(ropa):
                    self._driver.execute_script(
                        "window.scrollTo(0, document.body.scrollHeight)"
                    )
                    sleep(6)
                    ropa = self.obtener_publicaciones(
                        By.XPATH,
                        '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]',
                    )
                sleep(2)
                log(
                    INFO,
                    "-------------------------------------------------------------------",
                )

        del self._driver.requests
        self._tiempo.cantidad_real = i - e
        self._tiempo.num_error = e
        log(INFO, f"Se halló {e} errores")
        log(INFO, "Fin de la extraccion")

    def guardar_datos(
        self,
        dataset,
        filetype="Data",
        folder="Data//datos_obtenidos",
        filename="fb_data",
    ):
        """
        Guarda los datos o errores obtenidos durante la ejecución del scraper

        Parameters
        ----------
        dataset: dict
            Conjunto de datos extraídos por el scraper
        filetype: str
            Indica si la información proviene de los datos o de los errores
        folder: str
            Ruta del archivo
        filename: str
            Nombre del archivo

        Returns
        -------
        None
        """
        log(INFO, f"Guardando {filetype}")
        df_fb_mkp_ropa = DataFrame(dataset)

        if len(df_fb_mkp_ropa) == 0:
            log(
                INFO,
                f"El archivo de tipo {filetype} no se va a guardar por no tener información",
            )
            return

        if filetype == "Data":
            df_fb_mkp_ropa.drop(len(df_fb_mkp_ropa) - 1, axis=0, inplace=True)
            cantidad = len(df_fb_mkp_ropa)
            self._tiempo.cantidad = cantidad
        elif filetype == "Error":
            cantidad = self._tiempo.num_error
        else:
            log(
                INFO,
                f"El archivo de tipo {filetype} no está admitido. Solo se aceptan los valores Data y Error",
            )
            return

        datetime_obj = datetime.strptime(self._tiempo.fecha, "%d/%m/%Y")
        filepath = path.join(folder, datetime_obj.strftime("%d-%m-%Y"))
        filename = (
            filename
            + "_"
            + datetime_obj.strftime("%d%m%Y")
            + "_"
            + str(cantidad)
            + ".xlsx"
        )
        if not path.exists(filepath):
            makedirs(filepath)
        df_fb_mkp_ropa.to_excel(path.join(filepath, filename), index=False)
        log(INFO, f"{filetype} Guardados Correctamente")

    def guardar_tiempos(self, filename, sheet_name):
        """
        Guarda la información del tiempo de ejecución del scraper

        Parameters
        ----------
        filename: str
            Nombre del archivo
        sheet_name: str
            Nombre de la hoja de cálculo

        Returns
        -------
        None
        """
        log(INFO, "Guardando tiempos")
        self._tiempo.set_param_final()
        header_exist = True
        if path.isfile(filename):
            tiempos = load_workbook(filename)
            if sheet_name not in [ws.title for ws in tiempos.worksheets]:
                tiempos.create_sheet(sheet_name)
                header_exist = False
        else:
            tiempos = Workbook()
            tiempos.create_sheet(sheet_name)
            header_exist = False
        worksheet = tiempos[sheet_name]
        if not header_exist:
            keys = cambiar_posiciones(list(self._tiempo.__dict__.keys())[1:], 0, 1)
            worksheet.append(keys)
        values = cambiar_posiciones(list(self._tiempo.__dict__.values())[1:], 0, 1)
        worksheet.append(values)
        tiempos.save(filename)
        tiempos.close()
        log(INFO, "Tiempos Guardados Correctamente")

In [8]:
def config_log(
    log_folder, log_filename, log_file_mode, log_file_encoding, fecha_actual
):
    """
    Función que configura los logs para rastrear al programa
        Parameter:
                log_folder (str): Carpeta donde se va a generar el archivo log
                log_filename (str): Nombre del archivo log a ser generado
                fecha_actual (datetime): Fecha actual de la creación del archivo log
        Returns:
                None
    """
    seleniumLogger.setLevel(ERROR)
    urllibLogger.setLevel(ERROR)
    logger = getLogger("seleniumwire")
    logger.setLevel(ERROR)
    log_path = path.join(log_folder, fecha_actual.strftime("%d-%m-%Y"))
    log_filename = log_filename + "_" + fecha_actual.strftime("%d%m%Y") + ".log"
    if not path.exists(log_path):
        makedirs(log_path)
    basicConfig(
        format="%(asctime)s %(message)s",
        level=INFO,
        handlers=[
            StreamHandler(),
            FileHandler(
                path.join(log_path, log_filename), log_file_mode, log_file_encoding
            ),
        ],
    )


def validar_parametros(parametros):
    """
    Función que valida si los parámetros a usar están definidos
         Parameter:
                 parametros (list): Lista de parámetros

        Returns:
               None
    """
    for parametro in parametros:
        if not parametro:
            log(ERROR, "Parámetros incorrectos")
            return False
    log(INFO, "Parámetros válidos")
    return True


def cambiar_posiciones(lista, index1, index2):
    """
    Función que intercambia las posiciones de 2 elementos de un arreglo
         Parameter:
                 lista (list): Lista no vacía de elementos
                 index1 (int): Posición del primer elemento
                 index2 (int): Posición del segundo elemento

        Returns:
               list
    """
    if len(lista) == 0:
        return []
    aux = lista[index2]
    lista[index2] = lista[index1]
    lista[index1] = aux
    return lista

In [9]:
def main():
    try:
        # Formato para el debugger
        fecha_actual = datetime.now().date() - timedelta(days=1)
        config_log("Log", "fb_ropa_log", "w", "utf-8", fecha_actual)
        log(INFO, "Configurando Formato Básico del Debugger")

        # Cargar variables de entorno
        log(INFO, "Cargando Variables de entorno")
        load_dotenv()

        # Url de la categoría a scrapear
        url_ropa = getenv("URL_CATEGORY")

        # Parámetros para guardar la data extraída por el scraper
        data_filename = getenv("DATA_FILENAME")
        data_folder = getenv("DATA_FOLDER")

        # Parámetros para guardar la medición de la ejecución del scraper
        filename_tiempos = getenv("FILENAME_TIEMPOS")
        sheet_tiempos = getenv("SHEET_TIEMPOS")

        # Parámetros para guardar los errores durante la ejecución por el scraper
        error_filename = getenv("ERROR_FILENAME")
        error_folder = getenv("ERROR_FOLDER")

        # Validar parámetros
        if not validar_parametros(
            [
                url_ropa,
                data_filename,
                data_folder,
                filename_tiempos,
                sheet_tiempos,
                error_filename,
                error_folder,
            ]
        ):
            return

        # Inicializar scrapper
        scraper = ScraperFb(fecha_actual)

        # Iniciar sesión
        scraper.iniciar_sesion()

        # Extracción de datos
        scraper.mapear_datos(url_ropa)

        # Guardando la data extraída por el scraper
        scraper.guardar_datos(scraper.data.dataset, "Data", data_folder, data_filename)

        # Guardando los errores extraídos por el scraper
        scraper.guardar_datos(
            scraper.errores.errores, "Error", error_folder, error_filename
        )

        # Guardando los tiempos durante la ejecución del scraper
        scraper.guardar_tiempos(filename_tiempos, sheet_tiempos)
        log(INFO, "Programa finalizado")
    except Exception as error:
        log(ERROR, f"Error: {error}")
        log(INFO, "Programa ejecutado con fallos")

In [31]:
if __name__ == "__main__":
    main()

2023-02-02 02:33:37,813 Configurando Formato Básico del Debugger
2023-02-02 02:33:37,829 Cargando Variables de entorno
2023-02-02 02:33:37,829 Parámetros válidos
2023-02-02 02:33:37,829 Inicializando scraper
2023-02-02 02:33:37,829 Hora de inicio: 02:33:37
2023-02-02 02:33:37,829 ====== WebDriver manager ======
2023-02-02 02:33:38,642 Get LATEST chromedriver version for google-chrome 108.0.5359
2023-02-02 02:33:39,892 Driver [C:\Users\param\.wdm\drivers\chromedriver\win32\108.0.5359\chromedriver.exe] found in cache
2023-02-02 02:33:41,421 Iniciando sesión
2023-02-02 02:33:56,730 Inicio de sesión con éxito
2023-02-02 02:33:56,731 Accediendo a la URL
2023-02-02 02:34:19,196 Mapeando Publicaciones
2023-02-02 02:34:19,212 Creando variables
2023-02-02 02:34:19,212 Scrapeando item 1
2023-02-02 02:34:26,791 -------------------------------------------------------------------
2023-02-02 02:34:26,792 Scrapeando item 2
2023-02-02 02:34:34,051 ------------------------------------------------------

2023-02-02 02:41:15,410 -------------------------------------------------------------------
2023-02-02 02:41:15,411 Scrapeando item 57
2023-02-02 02:41:28,684 -------------------------------------------------------------------
2023-02-02 02:41:28,684 Scrapeando item 58
2023-02-02 02:41:35,895 -------------------------------------------------------------------
2023-02-02 02:41:35,895 Scrapeando item 59
2023-02-02 02:41:43,079 -------------------------------------------------------------------
2023-02-02 02:41:43,079 Scrapeando item 60
2023-02-02 02:41:50,260 -------------------------------------------------------------------
2023-02-02 02:41:50,260 Scrapeando item 61
2023-02-02 02:41:57,530 -------------------------------------------------------------------
2023-02-02 02:41:57,530 Scrapeando item 62
2023-02-02 02:42:04,930 -------------------------------------------------------------------
2023-02-02 02:42:04,946 Scrapeando item 63
2023-02-02 02:42:12,195 -------------------------------

2023-02-02 02:46:29,071 Scrapeando item 98
2023-02-02 02:46:34,217 SHORT TORERO BORDADO
2023-02-02 02:46:34,217 Item 98 scrapeado con éxito
2023-02-02 02:46:36,256 -------------------------------------------------------------------
2023-02-02 02:46:36,258 Scrapeando item 99
2023-02-02 02:46:41,431 Vestido Small SPRINGFIELD
2023-02-02 02:46:41,431 Item 99 scrapeado con éxito
2023-02-02 02:46:43,469 -------------------------------------------------------------------
2023-02-02 02:46:43,469 Scrapeando item 100
2023-02-02 02:46:48,632 Remató vestido de novia
2023-02-02 02:46:48,632 Item 100 scrapeado con éxito
2023-02-02 02:46:50,675 -------------------------------------------------------------------
2023-02-02 02:46:50,675 Scrapeando item 101
2023-02-02 02:46:55,837 Sandalias Para Bebé
2023-02-02 02:46:55,837 Item 101 scrapeado con éxito
2023-02-02 02:46:57,879 -------------------------------------------------------------------
2023-02-02 02:46:57,879 Scrapeando item 102
2023-02-02 02:47:

2023-02-02 02:50:39,873 Scrapeando item 132
2023-02-02 02:50:45,019 LINDOS BOTINES DE MODA PARA NIÑAS COLECCION OTOÑO-INVIERNO 2022
2023-02-02 02:50:45,019 Item 132 scrapeado con éxito
2023-02-02 02:50:47,064 -------------------------------------------------------------------
2023-02-02 02:50:47,064 Scrapeando item 133
2023-02-02 02:50:52,212 Carteras Pop It. Magdalena Líquidación
2023-02-02 02:50:52,212 Item 133 scrapeado con éxito
2023-02-02 02:50:54,256 -------------------------------------------------------------------
2023-02-02 02:50:54,256 Scrapeando item 134
2023-02-02 02:50:59,415 OFERTA de Polos para hombres de MARCA de EXPORTACION a precio de FABRICA
2023-02-02 02:50:59,415 Item 134 scrapeado con éxito
2023-02-02 02:51:01,447 -------------------------------------------------------------------
2023-02-02 02:51:01,447 Scrapeando item 135
2023-02-02 02:51:06,592 Sandalias de tiras talla 35 al 39
2023-02-02 02:51:06,592 Item 135 scrapeado con éxito
2023-02-02 02:51:08,629 ------

2023-02-02 02:54:54,708 ☺️😍 Zapatos De Mujer 👠👠👠💖🌹
2023-02-02 02:54:54,708 Item 165 scrapeado con éxito
2023-02-02 02:54:56,744 -------------------------------------------------------------------
2023-02-02 02:54:56,745 Scrapeando item 166
2023-02-02 02:55:01,902 Sandalias modernas 35-39
2023-02-02 02:55:01,902 Item 166 scrapeado con éxito
2023-02-02 02:55:03,937 -------------------------------------------------------------------
2023-02-02 02:55:03,939 Scrapeando item 167
2023-02-02 02:55:09,088 Zapatos De Cuero Para Niña - TALLA 28
2023-02-02 02:55:09,088 Item 167 scrapeado con éxito
2023-02-02 02:55:11,129 -------------------------------------------------------------------
2023-02-02 02:55:11,130 Scrapeando item 168
2023-02-02 02:55:16,264 conjunto elegante
2023-02-02 02:55:16,264 Item 168 scrapeado con éxito
2023-02-02 02:55:18,305 -------------------------------------------------------------------
2023-02-02 02:55:18,306 Scrapeando item 169
2023-02-02 02:55:23,450 Aquashoes
2023-0

...

2023-02-02 03:03:25,362 -------------------------------------------------------------------
2023-02-02 03:03:25,364 Scrapeando item 234
2023-02-02 03:03:30,524 LEGGIN AMERICANO ORIGINAL MARCA PINK
2023-02-02 03:03:30,524 Item 234 scrapeado con éxito
2023-02-02 03:03:32,565 -------------------------------------------------------------------
2023-02-02 03:03:32,567 Scrapeando item 235
2023-02-02 03:03:37,733 Enterizo Short
2023-02-02 03:03:37,733 Item 235 scrapeado con éxito
2023-02-02 03:03:39,763 -------------------------------------------------------------------
2023-02-02 03:03:39,764 Scrapeando item 236
2023-02-02 03:03:44,912 Lentes de sol para niñas
2023-02-02 03:03:44,912 Item 236 scrapeado con éxito
2023-02-02 03:03:46,950 -------------------------------------------------------------------
2023-02-02 03:03:46,951 Scrapeando item 237
2023-02-02 03:03:52,114 Fabrica de Camisas Hilton
2023-02-02 03:03:52,114 Item 237 scrapeado con éxito
2023-02-02 03:03:54,158 ---------------------

2023-02-02 03:07:35,076 Falvas Envolventes Largas 💃
2023-02-02 03:07:35,076 Item 267 scrapeado con éxito
2023-02-02 03:07:37,117 -------------------------------------------------------------------
2023-02-02 03:07:37,118 Scrapeando item 268
2023-02-02 03:07:42,323 Sandalias
2023-02-02 03:07:42,323 Item 268 scrapeado con éxito
2023-02-02 03:07:44,356 -------------------------------------------------------------------
2023-02-02 03:07:44,358 Scrapeando item 269
2023-02-02 03:07:49,541 Hermosos TOPS súper frescos
2023-02-02 03:07:49,543 Item 269 scrapeado con éxito
2023-02-02 03:07:51,567 -------------------------------------------------------------------
2023-02-02 03:07:51,569 Scrapeando item 270
2023-02-02 03:07:56,750 Conjunto
2023-02-02 03:07:56,751 Item 270 scrapeado con éxito
2023-02-02 03:07:58,779 -------------------------------------------------------------------
2023-02-02 03:07:58,780 Scrapeando item 271
2023-02-02 03:08:03,983 Tacones De Vestir en tallas y colores
2023-02-02 

2023-02-02 03:11:54,064 Sandalias
2023-02-02 03:11:54,064 Item 301 scrapeado con éxito
2023-02-02 03:11:56,099 -------------------------------------------------------------------
2023-02-02 03:11:56,100 Scrapeando item 302
2023-02-02 03:12:01,303 Sandalia de cuero
2023-02-02 03:12:01,303 Item 302 scrapeado con éxito
2023-02-02 03:12:03,352 -------------------------------------------------------------------
2023-02-02 03:12:03,353 Scrapeando item 303
2023-02-02 03:12:08,558 Casaca de béisbol RED SOX para niño T/ 7
2023-02-02 03:12:08,558 Item 303 scrapeado con éxito
2023-02-02 03:12:10,597 -------------------------------------------------------------------
2023-02-02 03:12:10,598 Scrapeando item 304
2023-02-02 03:12:15,812 Terno De Niño Blanco
2023-02-02 03:12:15,812 Item 304 scrapeado con éxito
2023-02-02 03:12:17,844 -------------------------------------------------------------------
2023-02-02 03:12:17,846 Scrapeando item 305
2023-02-02 03:12:23,037 Polos gapnuevos s-m
2023-02-02 03:

2023-02-02 03:16:06,860 Item 335 scrapeado con éxito
2023-02-02 03:16:08,898 -------------------------------------------------------------------
2023-02-02 03:16:08,900 Scrapeando item 336
2023-02-02 03:16:14,097 Sandalias
2023-02-02 03:16:14,097 Item 336 scrapeado con éxito
2023-02-02 03:16:16,126 -------------------------------------------------------------------
2023-02-02 03:16:16,128 Scrapeando item 337
2023-02-02 03:16:21,348 Vendo vestidos
2023-02-02 03:16:21,348 Item 337 scrapeado con éxito
2023-02-02 03:16:23,375 -------------------------------------------------------------------
2023-02-02 03:16:23,376 Scrapeando item 338
2023-02-02 03:16:28,622 🌸Vestido chalis xl en chalis premium 🌸
2023-02-02 03:16:28,622 Item 338 scrapeado con éxito
2023-02-02 03:16:30,651 -------------------------------------------------------------------
2023-02-02 03:16:30,652 Scrapeando item 339
2023-02-02 03:16:35,844 Casacas
2023-02-02 03:16:35,844 Item 339 scrapeado con éxito
2023-02-02 03:16:37,878

2023-02-02 03:20:27,344 Scrapeando item 370
2023-02-02 03:20:32,561 Shorer De Dama
2023-02-02 03:20:32,561 Item 370 scrapeado con éxito
2023-02-02 03:20:34,596 -------------------------------------------------------------------
2023-02-02 03:20:34,598 Scrapeando item 371
2023-02-02 03:20:39,846 Talla XXXXL, XXXL, XXL y XL. Blusas de Chalis. Contraentrega. Delivery a Toda Lima y Callao. Envios
2023-02-02 03:20:39,846 Item 371 scrapeado con éxito
2023-02-02 03:20:41,889 -------------------------------------------------------------------
2023-02-02 03:20:41,890 Scrapeando item 372
2023-02-02 03:20:47,127 Mango casaca mujer talla M
2023-02-02 03:20:47,127 Item 372 scrapeado con éxito
2023-02-02 03:20:49,158 -------------------------------------------------------------------
2023-02-02 03:20:49,160 Scrapeando item 373
2023-02-02 03:20:54,359 Botas de seguridad (punta de acero) para dama 👷‍♀️
2023-02-02 03:20:54,359 Item 373 scrapeado con éxito
2023-02-02 03:20:56,401 -----------------------